In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

from copy import deepcopy

In the game [peg solitaire](https://en.wikipedia.org/wiki/Peg_solitaire) you're given a starting position.  On each move, a peg is allowed to jump over another peg and land in an empty square.  The peg that was jumped over is removed. The goal is to have just one peg remaining. We'll consider a variant played on a rectangular grid.

# **Setup**

The following code handles all the game logic for you.  You don't need to understand the details, but you should read the docstring at the beginning to get a feel for how to use it.  

You shouldn't need to modify this, although you're welcome to.

(Note that this is not intended to be a particularly efficient or robust implementation.)

In [ ]:
class SolitaireGame:

  """
  Represents a game of Solitaire played out up to a given move, along with
  the move history.

  ...

  Methods
  -------
  legal_moves()
      Returns the full list of legal moves from the current board state

  make_move(move)
      Updates the current board by applying a given move
  
  undo()
      Undo the last move
  
  is_won()
      Returns true precisely when there's a single peg left

  ID()
      Gives a unique integer ID to the current board state (gotten by 
      interpreting the board state as an integer in binary with pegs 
      corresponding to 1's)
  """

  def __init__(self, init_board):
    """
    Creates a new game from an initial board given as a rectangular array.
    """
    self.history = [init_board]
    self.board = init_board
    self.rows = len(init_board)
    self.cols = len(init_board[0])
    for row in init_board:
      assert len(row) == self.cols, "Board must be rectangular."

  def legal_moves(self):
    '''Returns the list of legal moves.'''

    moves = []
    for r in range(self.rows):
      for c in range(self.cols):
        for dir in [(0,1),(0,-1),(1,0),(-1,0)]:
          if self._check_for_jump(r,c,dir):
            move = ((r,c), dir)
            moves.append(move)
    return moves

  def make_move(self, move):
    (r,c), dir = move    
    r1, c1 = r + dir[0],   c + dir[1]
    r2, c2 = r + 2*dir[0], c + 2*dir[1]

    new_board = deepcopy(self.board)

    new_board[r][c] = 0
    new_board[r1][c1] = 0
    new_board[r2][c2] = 1

    self.board = new_board
    self.history.append(new_board)

  def undo(self):
    self.history.pop(-1)
    self.board = self.history[-1]
  
  def is_won(self):
    pegs_left = sum([sum(row) for row in self.board])
    return pegs_left == 1
  
  def ID(self):
    id = 0
    for r in range(self.rows):
      for c in range(self.cols):
        id = 2 * id + self.board[r][c]
    return id

  def _check_for_jump(self, r,c, dir):
    '''check if the peg at position (r,c) can jump in the direction dir
    dir is either (0,1), (0,-1), (1,0) or (-1,0)'''

    r1, c1 = r + dir[0],   c + dir[1]
    r2, c2 = r + 2*dir[0], c + 2*dir[1]
    stays_inbounds = r2 < self.rows and c2 < self.cols and r2 >= 0 and c2 >= 0
    
    return stays_inbounds and self.board[r][c]==1 \
      and self.board[r1][c1]==1 and self.board[r2][c2]==0

In [ ]:
def game_animation(game):

  def make_frame(i, ax, game):
    ax.clear()
    board = game.history[i]

    ax.set_xlim([-0.5, game.cols-0.5])
    ax.set_ylim([-0.5, game.rows-0.5])
    ax.set_aspect('equal')
    ax.invert_yaxis()
    ax.set_xticks(range(game.cols))
    ax.set_yticks(range(game.rows))

    for r in range(game.rows):
      for c in range(game.cols):
        if board[r][c] == 1:
          ax.scatter(c,r, marker='o',s=1000, color='black')
        else:
          ax.scatter(c,r, marker='o',s=20, color='black')

  fig, ax = plt.subplots()
  ani = FuncAnimation(fig, make_frame, fargs=(ax, game) ,frames=range(len(game.history)),interval=1000)
  html = ani.to_jshtml()
  return html

# **Example Code**

Here's an example of how to use the code above.  Here we just keep making the first legal move while there's still a move left.  Then we create an animation for the game and display it.

In [ ]:
%%capture

board = [[1,0,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1]]
game = SolitaireGame(board)

while game.legal_moves(): #Same as "while len(game.legal_moves()) != 0"
  game.make_move(game.legal_moves()[0])

animation = game_animation(game)

In [ ]:
HTML(animation)

# **Solving Solitaire**

Now write some code to search for a winning sequence of moves using a version of DFS.  Your function should return ```True``` if there is a winning sequence of moves and ```False``` otherwise.  You'll modify ```game``` as you go.  You don't need to return ```game```, just stop modifying it once you're in a winning position.  To get to a winning position you should use ```game.legal_moves()``` to get all legal moves and ```game.make_move()``` to apply a given move.  When you get stuck, you can call ```game.undo()```.  You're welcome to add other functions if it's helpful.  The code should be pretty short. Take a look at the pseudocode for DFS in the textbook if you're stuck.

In [ ]:
def solve(game):
  if game.is_won():
    return True
  for move in game.legal_moves():
    game.make_move(move)
    if solve(game):
      return True
    else:
      game.undo()
  return False

**Here are a couple quick test to try out your code and visualize the solution.**

Test 1.

In [ ]:
board = [[1,0,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1]]
game = SolitaireGame(board)
solvable = solve(game)
print(f"Solution exists: {solvable}")

Solution exists: True


In [ ]:
%%capture
animation = game_animation(game);

In [ ]:
HTML(animation)

Test 2.

In [ ]:
board = [[0,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1]]
game = SolitaireGame(board)
solvable = solve(game)
print(f"Solution exists: {solvable}")

Solution exists: False


**Add your own test.  How big of a board can you get it to work for?**

Most 4\*4 boards can be figured out within 1 minutes. However, for 5\*5 boards, it generally takes more than 10 minutes.

In [ ]:
board = [[0, 0, 1, 1, 1],
         [0, 0, 1, 1, 0],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 0, 0],
         [1, 0, 1, 0, 0]]
game = SolitaireGame(board)
solvable = solve(game)
print(f"Solution exists: {solvable}")

Solution exists: False
